In [118]:
%matplotlib inline
from collections import Counter

# members = pd.read_csv("data/members_v3.csv")
# train = pd.read_csv("data/train_v2.csv")
# transactions = pd.read_csv("data/transactions_v2.csv")
# user_logs = pd.read_csv("data/user_logs_v2.csv")
# sample_submission = pd.read_csv("data/sample_submission_v2.csv")
train_final = pd.read_csv("data/train_final.csv")
test_final = pd.read_csv("data/test_final.csv")

# user_logs
 - id별로 sum을 하고 기간을 date column 삭제u

# members
 - gender의 NaN 값은 확률로 fill
 - bd의 이상값들은 가정을 바탕으로 fill

# transaction
 - transactio_date가 최신인 값을 기준으로 중복제거
 - train에서 생기는 NaN데이터는 삭제

In [100]:
train_final.shape, test_final.shape

((933578, 22), (907471, 22))

In [101]:
train_final.head()

,msno,is_churn,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,...,num_75,num_985,num_100,num_unq,total_secs,city,gender,registered_via,registration_init_time,age
0,f/NmvEzHfhINFEYZTR05prUdr+E+3+oewvweYz9cCQE=,1,36.0,30.0,180.0,180.0,0.0,20170311.0,20170411.0,0.0,...,2.000000,0.000000,26.000000,30.00000,6986.509000,13.0,male,3.0,20131223.0,20.0
1,zLo9f73nGGT1p21ltZC3ChiRnAVvgibMyazbCxvWPcg=,1,15.0,90.0,300.0,300.0,0.0,20170314.0,20170615.0,0.0,...,32.000000,22.000000,205.000000,432.00000,67810.467000,13.0,male,3.0,20131227.0,18.0
2,8iF/+8HY8lJKFrTc7iR9ZYGCG2Ecrogbc2Vy5YhsfhQ=,1,41.0,30.0,149.0,149.0,1.0,20151208.0,20180108.0,0.0,...,15.659063,17.442416,483.033693,472.74419,126291.230177,1.0,female,7.0,20140109.0,38.0
3,K6fja4+jmoZ5xG6BypqX80Uw/XKpMgrEMdG2edFOxnA=,1,41.0,30.0,99.0,99.0,1.0,20170316.0,20170918.0,0.0,...,4.000000,4.000000,962.000000,548.00000,239882.241000,13.0,female,7.0,20140125.0,35.0
4,ibIHVYBqxGwrSExE63/omeDD99M5vYB3CN2HzkEY+eM=,1,40.0,30.0,149.0,149.0,1.0,20170316.0,20170314.0,1.0,...,1.000000,4.000000,37.000000,48.00000,10110.665000,22.0,female,3.0,20140126.0,26.0


In [102]:
test_final.head()

,msno,is_churn,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,...,num_75,num_985,num_100,num_unq,total_secs,city,gender,registered_via,registration_init_time,age
0,4n+fXlyJvfQnTeKXTWT507Ll4JVYGrOC8LHCfwBmPE4=,0,41.0,30.0,99.0,99.0,1.0,20170318.0,20170418.0,0.0,...,16.633033,18.838576,541.108209,515.732526,141113.079095,1.0,female,7.0,20150718.0,31.0
1,aNmbC1GvFUxQyQUidCVmfbQ0YeCuwkPzEdQ0RwWyeZM=,0,34.0,30.0,149.0,149.0,1.0,20170331.0,20170430.0,0.0,...,125.000000,131.000000,1701.000000,2273.000000,500802.640000,4.0,male,9.0,20051030.0,28.0
2,rFC9eSG/tMuzpre6cwcMLZHEYM89xY02qcz7HL4//jc=,0,41.0,30.0,99.0,99.0,1.0,20170315.0,20170415.0,0.0,...,1.000000,2.000000,84.000000,149.000000,23814.270000,4.0,male,7.0,20141101.0,34.0
3,WZ59dLyrQcE7ft06MZ5dj40BnlYQY7PHgg/54+HaCSE=,0,41.0,30.0,99.0,99.0,1.0,20170327.0,20170427.0,0.0,...,16.633033,18.838576,541.108209,515.732526,141113.079095,1.0,female,9.0,17.0,22.0
4,aky/Iv8hMp1/V/yQHLtaVuEmmAxkB5GuasQZePJ7NU4=,0,30.0,30.0,129.0,129.0,1.0,20170322.0,20170421.0,0.0,...,16.633033,18.838576,541.108209,515.732526,141113.079095,1.0,female,13.0,20161222.0,32.0


In [103]:
train_final['gender'] = train_final['gender'].replace({'male':0, 'female':1})

In [156]:
a = train_final['plan_list_price'] - train_final['actual_amount_paid']
count = 0
for i in a:
    if i != 0:
        count += 1
        
count # discount를 받은 것으로 추정

1712

In [106]:
date_cols = ['transaction_date', 'membership_expire_date', 'registration_init_time']
print (train_final[date_cols].dtypes)

c = train_final['transaction_date'].values
for i in c:
    int(i)
    
train_final['transaction_date'] = c

d = train_final['membership_expire_date'].values
for j in d:
    int(j)
    
train_final['membership_expire_date'] = d

transaction_date          float64
membership_expire_date    float64
registration_init_time    float64
dtype: object


In [140]:
# float64 type의 date column들을 datetime type으로 변환
train_final['transaction_date'] = pd.to_datetime(train_final['transaction_date'], format='%Y%m%d')
train_final['membership_expire_date'] = pd.to_datetime(train_final['membership_expire_date'], format='%Y%m%d')

In [157]:
train_final[train_final['registration_init_time'] < 2000] # registration_init_time의 이상값 발견
                                                          # 108210개

,msno,is_churn,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,...,num_75,num_985,num_100,num_unq,total_secs,city,gender,registered_via,registration_init_time,age
20,NTOh3fAZrLoIyE1lxoRpp2lzEraNnnWbUvOhrmO1bUM=,1,41.0,30.0,149.0,149.0,1.0,2015-12-19,2017-05-20,0.0,...,15.659063,17.442416,483.033693,472.744190,126291.230177,13.0,female,3.0,26.0,50.0
39,+wisdxP166qHezQOKLPv4l4AfZClRF3ExWGD2rzYYCU=,1,41.0,30.0,149.0,149.0,1.0,2017-03-06,2017-03-06,1.0,...,15.659063,17.442416,483.033693,472.744190,126291.230177,1.0,male,9.0,26.0,27.0
49,P9FjH43/pjIS6PwcXP8VcA+P4RTq/mH+F3DLuXDpFkE=,1,41.0,30.0,99.0,99.0,1.0,2017-03-22,2017-03-22,1.0,...,15.659063,17.442416,483.033693,472.744190,126291.230177,4.0,female,9.0,29.0,23.0
53,Ybp7mbACy/qnytzEr8rZykiEyL23ESsAIZRo17VAT/8=,1,41.0,30.0,99.0,99.0,1.0,2017-03-11,2017-03-11,1.0,...,15.659063,17.442416,483.033693,472.744190,126291.230177,13.0,female,7.0,34.0,40.0
78,r0ND2OhGFAUltb62mwbwM7W9nnI+MMA3IAdBHk3abbI=,1,41.0,30.0,99.0,99.0,1.0,2017-03-20,2017-03-20,1.0,...,15.659063,17.442416,483.033693,472.744190,126291.230177,13.0,female,4.0,28.0,19.0
88,YOF2Z5cnRQQzJ0+jO1k+0nkQ5rO2KDZA1x+XbZ2vPaw=,1,41.0,30.0,99.0,99.0,1.0,2017-03-07,2017-03-07,1.0,...,15.659063,17.442416,483.033693,472.744190,126291.230177,6.0,male,9.0,29.0,52.0
89,e0N3u3laGuzrocxSIezxFVU+uNuiiN4RAAhr36rmV0Y=,1,41.0,30.0,99.0,99.0,1.0,2017-03-10,2017-03-10,1.0,...,15.659063,17.442416,483.033693,472.744190,126291.230177,1.0,male,7.0,27.0,28.0
96,47WmILK1vHIz0iCDMX1elQNorVhh1aupLKq1R5ddBto=,1,41.0,30.0,99.0,99.0,1.0,2017-03-12,2017-03-12,1.0,...,15.659063,17.442416,483.033693,472.744190,126291.230177,15.0,female,9.0,25.0,29.0
117,8Yfas8KHQ5UHsRwQ3Iesv2UXZ9FStQxCZ+TKaJ7Q+SA=,1,30.0,30.0,129.0,129.0,1.0,2017-03-03,2017-03-03,1.0,...,15.659063,17.442416,483.033693,472.744190,126291.230177,1.0,female,9.0,19.0,45.0
127,ZpboqV+244iMAeyKObEcf057ZQlDI5ndU9aqq6dABKc=,1,41.0,30.0,149.0,149.0,1.0,2017-03-07,2017-03-07,1.0,...,15.659063,17.442416,483.033693,472.744190,126291.230177,1.0,female,4.0,37.0,29.0


In [158]:
train_final.head()

,msno,is_churn,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,...,num_75,num_985,num_100,num_unq,total_secs,city,gender,registered_via,registration_init_time,age
0,f/NmvEzHfhINFEYZTR05prUdr+E+3+oewvweYz9cCQE=,1,36.0,30.0,180.0,180.0,0.0,2017-03-11,2017-04-11,0.0,...,2.000000,0.000000,26.000000,30.00000,6986.509000,13.0,male,3.0,20131223.0,20.0
1,zLo9f73nGGT1p21ltZC3ChiRnAVvgibMyazbCxvWPcg=,1,15.0,90.0,300.0,300.0,0.0,2017-03-14,2017-06-15,0.0,...,32.000000,22.000000,205.000000,432.00000,67810.467000,13.0,male,3.0,20131227.0,18.0
2,8iF/+8HY8lJKFrTc7iR9ZYGCG2Ecrogbc2Vy5YhsfhQ=,1,41.0,30.0,149.0,149.0,1.0,2015-12-08,2018-01-08,0.0,...,15.659063,17.442416,483.033693,472.74419,126291.230177,1.0,female,7.0,20140109.0,38.0
3,K6fja4+jmoZ5xG6BypqX80Uw/XKpMgrEMdG2edFOxnA=,1,41.0,30.0,99.0,99.0,1.0,2017-03-16,2017-09-18,0.0,...,4.000000,4.000000,962.000000,548.00000,239882.241000,13.0,female,7.0,20140125.0,35.0
4,ibIHVYBqxGwrSExE63/omeDD99M5vYB3CN2HzkEY+eM=,1,40.0,30.0,149.0,149.0,1.0,2017-03-16,2017-03-14,1.0,...,1.000000,4.000000,37.000000,48.00000,10110.665000,22.0,female,3.0,20140126.0,26.0
